In [1]:
# %pip install mysql-connector-python

import mysql.connector
import pandas as pd

#### Connection and Show Tables

In [2]:
# Check connection to Mysql
try:
    conn = mysql.connector.connect(
        host="***",  
        user="***",
        password="***",  
        database="product_sales_db",
        port=3306
    )
    print("✅ Connected to MySQL successfully!")
    conn.close()
except mysql.connector.Error as err:
    print(f"❌ MySQL Error: {err}")


✅ Connected to MySQL successfully!


In [3]:
# Connect to MySQL database
conn = mysql.connector.connect(
    host="***",  
    user="***",
    password="***",  
    database="product_sales_db",
    port=3306
)
cursor = conn.cursor()

# Function to fetch and display tables
def fetch_table(table_name):
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql(query, conn)
    return df

# Fetch all tables
tables = ["product_classes", "products", "promotions", "sales"]
for table in tables:
    print(f"\n {table.upper()} Table \n")
    display(fetch_table(table))


 PRODUCT_CLASSES Table 



,product_class_id,product_subcategory,product_category,product_department,product_family
0,1,Cereal,Food,Grocery,Breakfast
1,2,Beverage,Drinks,Grocery,Soft Drinks
2,3,Dairy,Food,Refrigerated,Milk Products
3,4,Tissue,Daily,Grocery,Daily Use



 PRODUCTS Table 



,product_id,product_class_id,brand_name,product_name,is_low_fat_flg,is_recyclable_flg,gross_weight,net_weight
0,101,1,Kellogg,Corn Flakes,1,1,500.0,450.0
1,102,2,Coca-Cola,Coke Zero,1,1,350.0,330.0
2,103,3,DairyPure,Whole Milk,0,1,1000.0,950.0
3,104,4,Denny,Tissue,0,1,18.6,16.6



 PROMOTIONS Table 



,promotion_id,promotion_name,media_type,cost,start_date,end_date
0,201,Summer Sale,TV,50.0,2024-06-01,2024-06-30
1,202,Holiday Discount,Online,3000.0,2024-12-01,2024-12-31
2,203,Back to School,Radio,200.0,2024-08-01,2024-08-15
3,204,Black Monday,Grocery,10000.0,2024-11-01,2024-11-02



 SALES Table 



,product_id,store_id,customer_id,promotion_id,store_sales,store_cost,units_sold,transaction_date
0,101,1,1001,201.0,50.0,30.0,10.0,2024-06-05
1,102,2,1002,202.0,100.0,60.0,20.0,2024-12-10
2,103,1,1003,203.0,75.0,40.0,15.0,2024-08-05
3,101,3,1004,NaN,40.0,25.0,8.0,2024-07-01
4,102,2,1005,201.0,90.0,50.0,18.0,2024-06-20
5,102,1,1001,201.0,50.0,30.0,2.0,2024-02-25
6,103,2,1001,201.0,75.0,40.0,3.0,2024-02-26
7,102,3,1002,202.0,100.0,60.0,5.0,2024-02-27
8,101,1,1003,201.0,120.0,70.0,4.0,2024-02-28


### The VP of Sales feels that some product categories don't sell and can be completely removed from the inventory.
Find out What percentage of product categories have never been sold.

In [4]:
query = """
SELECT (100.0 * (COUNT(DISTINCT pc.product_category) - COUNT(DISTINCT spc.product_category)) 
        / COUNT(DISTINCT pc.product_category)
    ) AS pct_product_categories_never_sold
FROM product_classes pc
LEFT JOIN (
    SELECT DISTINCT pc.product_category
    FROM sales s
    JOIN products p ON s.product_id = p.product_id
    JOIN product_classes pc ON p.product_class_id = pc.product_class_id
) spc ON pc.product_category = spc.product_category;

"""

df1 = pd.read_sql(query, conn)
df1

,pct_product_categories_never_sold
0,33.33333


Since the percentage of unsold product categories is quite high, the VP may consider taking action to remove some of them.

### The company wants to optimize its product offerings by identifying the top-performing products.
Which products generate the highest revenue?

In [5]:
query = """
SELECT p.product_name, SUM(s.store_sales) AS total_revenue
FROM sales s
JOIN products p ON s.product_id = p.product_id
GROUP BY p.product_name
ORDER BY total_revenue DESC
LIMIT 1;
"""

df2 = pd.read_sql(query, conn)
df2

,product_name,total_revenue
0,Coke Zero,340.0


The analysis reveals that Coke Zero generates the highest revenue. The company may consider prioritizing it for promotions, ensuring adequate stock levels, and exploring ways to further capitalize on their success.

###  The marketing team wants to assess the effectiveness of various promotions to determine which strategies drive the highest sales increase. 
Which promotions resulted in the highest sales increase?

In [6]:
query = """
SELECT pr.promotion_name, SUM(s.store_sales) AS total_sales
FROM sales s
JOIN promotions pr ON s.promotion_id = pr.promotion_id
GROUP BY pr.promotion_name
ORDER BY total_sales DESC;

"""

df3 = pd.read_sql(query, conn)
df3

,promotion_name,total_sales
0,Summer Sale,385.0
1,Holiday Discount,200.0
2,Back to School,75.0


The company may consider focusing on the Summer Sale and refining or discontinuing less effective ones.

### The company wants to evaluate the financial effectiveness of different promotions. Identifying promotions where costs exceeded revenue ensures that inefficient campaigns are either improved or discontinued.
Which promotions had the highest ROI? Are there promotions where the cost exceeded revenue?

In [7]:
query = """
WITH promo_sales AS (
    SELECT s.promotion_id, SUM(s.store_sales) AS total_revenue
    FROM sales s
    WHERE s.promotion_id IS NOT NULL
    GROUP BY s.promotion_id
)
SELECT 
    p.promotion_id, 
    p.promotion_name, 
    p.cost AS promotion_cost, 
    IFNULL(ps.total_revenue, 0) AS revenue_generated,
    (IFNULL(ps.total_revenue, 0) - p.cost) AS net_profit,
    ROUND((IFNULL(ps.total_revenue, 0) / NULLIF(p.cost, 0)), 2) AS ROI
FROM promotions p
LEFT JOIN promo_sales ps ON p.promotion_id = ps.promotion_id
ORDER BY ROI DESC;

"""

df4 = pd.read_sql(query, conn)
df4

,promotion_id,promotion_name,promotion_cost,revenue_generated,net_profit,ROI
0,201,Summer Sale,50.0,385.0,335.0,7.70
1,203,Back to School,200.0,75.0,-125.0,0.38
2,202,Holiday Discount,3000.0,200.0,-2800.0,0.07
3,204,Black Monday,10000.0,0.0,-10000.0,0.00


The company may consider reallocating budget toward high-performing promotions and reworking or eliminating unprofitable ones to maximize overall profitability.

### Understanding the number of one-time customers helps the company assess customer retention and loyalty. 
How many customers only made a single purchase?

In [8]:
query = """
SELECT COUNT(customer_id) AS single_purchase_customers
FROM (
    SELECT customer_id
    FROM sales
    GROUP BY customer_id
    HAVING COUNT(*) = 1
) AS single_purchases;
"""

df5 = pd.read_sql(query, conn)
df5

,single_purchase_customers
0,2


How many percentage of customers only made a single purchase?

In [9]:
query = """
SELECT 
    (COUNT(single_purchases.customer_id) * 100.0 / COUNT(DISTINCT sales.customer_id)) AS single_purchase_percentage
FROM (
    SELECT customer_id
    FROM sales
    GROUP BY customer_id
    HAVING COUNT(*) = 1
) AS single_purchases
RIGHT JOIN sales ON single_purchases.customer_id = sales.customer_id;

"""

df6 = pd.read_sql(query, conn)
df6

,single_purchase_percentage
0,40.0


The company may consider implementing strategies such as personalized marketing, loyalty programs, or follow-up engagement to encourage repeat purchases and improve customer retention.

### The company wants to evaluate the value of repeat customers compared to one-time buyers. Understanding their spending behavior can help shape retention strategies, loyalty programs, and customer engagement efforts to maximize revenue from high-value customers.
How does repeat purchases customer spending compare to single-purchase customers?

In [10]:
query = """
WITH customer_purchases AS (
    SELECT customer_id, COUNT(*) AS total_purchases, SUM(store_sales) AS total_spent
    FROM sales
    GROUP BY customer_id
)
SELECT 
    CASE 
        WHEN total_purchases = 1 THEN 'Single Purchase'
        WHEN total_purchases BETWEEN 2 AND 5 THEN 'Occasional Buyer'
        ELSE 'Frequent Buyer'
    END AS customer_segment,
    COUNT(customer_id) AS customer_count,
    AVG(total_spent) AS avg_spent_per_customer
FROM customer_purchases
GROUP BY customer_segment
ORDER BY avg_spent_per_customer DESC;

"""

df8 = pd.read_sql(query, conn)
df8

,customer_segment,customer_count,avg_spent_per_customer
0,Occasional Buyer,3,190.0
1,Single Purchase,2,65.0


The analysis shows that repeat customers tend to spend more than single-purchase customers. The company may consider investing in customer retention strategies, such as targeted promotions or loyalty rewards, to encourage repeat purchases and enhance long-term profitability.

In [11]:
# Close connection
cursor.close()
conn.close()